In [22]:
import pandas as pd
train=pd.read_csv('train.csv')
train

,ID,Filename,Label,ClassName
0,16257,AnnualCrop/AnnualCrop_142.jpg,0,AnnualCrop
1,3297,HerbaceousVegetation/HerbaceousVegetation_2835...,2,HerbaceousVegetation
2,17881,PermanentCrop/PermanentCrop_1073.jpg,6,PermanentCrop
3,2223,Industrial/Industrial_453.jpg,4,Industrial
4,4887,HerbaceousVegetation/HerbaceousVegetation_1810...,2,HerbaceousVegetation
...,...,...,...,...
18895,4498,HerbaceousVegetation/HerbaceousVegetation_1952...,2,HerbaceousVegetation
18896,1149,Pasture/Pasture_1252.jpg,5,Pasture
18897,15489,AnnualCrop/AnnualCrop_2332.jpg,0,AnnualCrop
18898,6287,Residential/Residential_332.jpg,7,Residential


In [23]:
num_classes=train['ClassName'].nunique()
print(num_classes)

10


In [6]:
from torch.utils.data import Dataset,DataLoader
from PIL import Image
from torchvision import transforms
class CustomDataset(Dataset):
    def __init__(self,path,transform):
        self.data=pd.read_csv(path)
        self.transform=transform
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_path=self.data.iloc[index,1]
        image=Image.open(img_path).convert('RGB')
        label=self.data.iloc[index,2]
        classname=self.data.iloc[index,3]
        
        if self.transform:
            image=self.transform(image)
            
        return image,label,classname
    
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

path='train.csv'
train=CustomDataset(path,transform)

c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\KARAN\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_py

In [7]:
train_loader=DataLoader(train,batch_size=64,shuffle=True)
train_loader

In [8]:
import timm
model=timm.create_model('efficientnet_b0',pretrained=True)

In [9]:
model

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [10]:
import torch
import torch.nn as nn

in_feats=model.classifier.in_features
model.classifier=nn.Linear(in_feats,num_classes)
print(model)

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

In [11]:
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=20, gamma=0.1)

In [8]:
path='validation.csv'
val=CustomDataset(path,transform)
val_loader=DataLoader(train,batch_size=32,shuffle=True)

path='test.csv'
test=CustomDataset(path,transform)
test_loader=DataLoader(test,batch_size=64,shuffle=False)

In [9]:
from tqdm import tqdm
import numpy as np
num_epochs=100
device='cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

train_losses=[]
val_loss_min=np.Inf

for epoch in tqdm(range(num_epochs)):
    train_loss = 0.0
    valid_loss = 0.0
    model.train()
    for _,(images,labels,_) in enumerate(tqdm(train_loader,desc=f'Epoch {epoch+1}')):
        images,labels=images.to(device),labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*images.size(0)
    model.eval()
    for images,labels,_ in val_loader:
        images,labels=images.to(device),labels.to(device)
        output = model(images)
        loss = criterion(output, labels)
        valid_loss += loss.item()*images.size(0)
    
    train_loss = train_loss/len(train_loader.dataset)
    val_loss = valid_loss/len(val_loader.dataset)
    train_losses.append(train_loss)
        
    print(f'\nTraining Loss: {train_loss:.4f} \tValidation Loss: {val_loss:.4f}')
    if val_loss <= val_loss_min:
        torch.save(model.state_dict(), 'effnet.pt')
        val_loss_min = val_loss

  0%|          | 0/2 [00:08<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 46.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 3.33 GiB is allocated by PyTorch, and 111.69 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import torch
from sklearn.metrics import classification_report

def test(model, test_loader, criterion, device, num_classes):
    model.eval()
    class_correct = [0.0] * num_classes
    class_total = [0.0] * num_classes
    test_loss = 0.0
    correct = 0
    total = 0
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            test_loss += loss.item() * images.size(0)
            
            _, predicted = torch.max(output, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            
            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += (predicted[i] == labels[i]).item()
                class_total[label] += 1
    
    test_loss /= len(test_loader.dataset)
    print(f'Test Loss: {test_loss:.4f}')

    for i in range(num_classes):
        if class_total[i] > 0:
            print(f'Accuracy of class {i}: {100 * class_correct[i] / class_total[i]:.2f}%')
    
    overall_accuracy = 100 * correct / total
    print(f'Overall Accuracy: {overall_accuracy:.2f}%')
    
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=[f'Class {i}' for i in range(num_classes)]))

best_model = timm.create_model('efficientnet_b0', pretrained=True)
in_feats = best_model.classifier.in_features
best_model.classifier = nn.Linear(in_feats, num_classes)
best_model.load_state_dict(torch.load('effnet.pt'))
best_model.to(device)

test(best_model, test_loader, criterion, device, num_classes)